In [36]:
from rhp_plan import plan
from pathlib import Path
import openpyxl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from copy import deepcopy


xlsx_file = Path('nrio_sut_181108.xlsx')
main_sheet = openpyxl.load_workbook(xlsx_file)['2008']
xlsx_file = Path('posternas_namn.xlsx')
shorthand_sheet = openpyxl.load_workbook(xlsx_file)['SUP10']

def zdivide(x, y):
    return np.divide(x, y, out=np.zeros_like(x), where=y!=0)

# Generates a 2D list of values from an xlsx file, reading left to right, top to bottom.
class Sheet:
    def __init__(self, sheet, left: int, top: int, right: int, bottom: int):
        result_sheet = []
        coordinates = []
        for row in sheet.iter_rows(min_row=left, min_col=top, max_row=right, max_col=bottom):
            data = []
            pos = []
            for cell in row:
                data.append(cell.value)
                pos.append(cell)

            result_sheet.append(data)
            coordinates.append(pos)

        self.result = result_sheet
        self.coordinates = coordinates

    @property
    def np_array(self):
        return np.array(self.result, dtype=np.float64)


class ProductionMatrix:
    def __init__(self, use_domestic: Sheet, use_imported: Sheet, supply: Sheet, other_variables: Sheet):
        self.use_domestic = use_domestic
        self.use_imported = use_imported
        self.supply = supply
        self.other_variables = other_variables

    @property
    def use_domestic_matrix(self):
        return np.matrix(np.concatenate([self.use_domestic.np_array, [[0] * len(self.use_domestic.np_array[0])]], axis=0)) # Add row at the end with zero values

    @property
    def use_imported_matrix(self):
        return np.matrix(np.concatenate([self.use_imported.np_array, [[0] * len(self.use_imported.np_array[0])]], axis=0))

    @property
    def supply_matrix(self):
        return np.matrix(np.concatenate([self.supply.np_array, [self.other_variables.np_array[-1]]])) # Add row at the end with copied values

    @property
    def other_variables_matrix(self):
        return np.matrix(self.other_variables.np_array)

    @property
    def worked_hours(self):
        return self.other_variables.np_array[-1] # TODO 

    def generate_matrix(self):
        data = self.supply_matrix - (self.use_domestic_matrix + self.use_imported_matrix)
        data = zdivide(data, self.worked_hours)

        return data


class OutputTarget:
    def __init__(self, final_demand_domestic: Sheet, final_demand_imported: Sheet):
        self.final_demand_domestic = final_demand_domestic
        self.final_demand_imported = final_demand_imported

    @property
    def final_demand_domestic_matrix(self):
        return np.matrix(self.final_demand_domestic.np_array)

    @property
    def final_demand_imported_matrix(self):
        return np.matrix(self.final_demand_imported.np_array)

    def generate_matrix(self):
        final_demand_domestic = self.final_demand_domestic_matrix.sum(axis=1, dtype=np.float64)
        final_demand_imported = self.final_demand_imported_matrix.sum(axis=1, dtype=np.float64)

        data = final_demand_domestic + final_demand_imported
        data[40][0] = 0.
        return np.append(data, [[0]], axis=0) 



production_matrix = ProductionMatrix(Sheet(main_sheet, 4, 3, 62, 61), Sheet(main_sheet, 94, 3, 152, 61), Sheet(main_sheet, 159, 3, 217, 61), Sheet(main_sheet, 69, 3, 87, 61))
output_target = OutputTarget(Sheet(main_sheet, 4, 68, 62, 74), Sheet(main_sheet, 94, 68, 152, 74))
export_vector = Sheet(main_sheet, 4, 75, 62, 75).np_array + Sheet(main_sheet, 94, 75, 152, 75).np_array
shorthand = [[x[0][4:] if x[0].startswith('CPA_') else x[0], x[1]] for x in Sheet(shorthand_sheet, 7, 2, 65, 3).result]

sector = Sheet(main_sheet, 3, 3, 3, 61)

p_res = production_matrix.generate_matrix()
o_res = output_target.generate_matrix()
worked_hours = production_matrix.worked_hours
import_matrix = zdivide(production_matrix.use_imported_matrix, worked_hours)

one_vector_59 = np.array([1] * p_res.shape[1])

[print(f'{short}: {desc}') for short, desc in shorthand]

time_steps_input = 3
planning_horizon_input = 3
step_horizon = time_steps_input + planning_horizon_input
supply_list_input, use_dom_list_input, use_imp_list_input, target_out_list_input, export_vector_list_input, export_prices_list_input, imp_prices_list_input = [], [], [], [], [], [], []
exp_vec = np.array(deepcopy(np.append(export_vector, [0]))).reshape([-1,1])

for T in range(step_horizon):
    supply_list_input.append(zdivide(production_matrix.supply_matrix, production_matrix.worked_hours))
    use_dom_list_input.append(zdivide(production_matrix.use_domestic_matrix, production_matrix.worked_hours))
    use_imp_list_input.append(zdivide(production_matrix.use_imported_matrix, production_matrix.worked_hours))
    #supply_list_input[T][:,40] = np.zeros(supply_list_input[T].shape[0]).reshape([-1,1])
    # use_dom_list_input[T][:,40] = 0.
    # use_imp_list_input[T][:,40] = 0.
    target_out_list_input.append(o_res)
    export_vector_list_input.append(exp_vec)
    export_prices_list_input.append(np.array([[1] for i in range(exp_vec.shape[0])]))
    imp_prices_list_input.append(np.array([[1] for i in range(exp_vec.shape[0])]))

depreciation_list_input = []
for T in range(step_horizon + 1):
    depreciation_list_input.append(np.matrix(np.eye(N=production_matrix.supply_matrix.shape[0], M=production_matrix.supply_matrix.shape[0], k=0)))

sector_name_input = deepcopy(sector.result[0])

sector_with_all_outputs_input = deepcopy(sector_name_input)
sector_with_all_outputs_input.append('CO2')


primary_resource_list_input = []
primary_resource_len = deepcopy((planning_horizon_input * supply_list_input[0].shape[1])) 
for T in range(planning_horizon_input + time_steps_input + 1):
    primary_resource_list_input.append(np.array([[1] for i in range(primary_resource_len)]))

plan_outcome = plan(time_steps_input, planning_horizon_input, primary_resource_list_input, supply_list_input, use_dom_list_input, use_imp_list_input,
     depreciation_list_input, target_out_list_input, export_vector_list_input, export_prices_list_input, imp_prices_list_input)


# plan details
result_list = plan_outcome[0]
lagrange_list = plan_outcome[1]
overshoot = []

atarget_output_aggregated_list = []
afinal_production_matrix_list = []
for T in range(planning_horizon_input + time_steps_input): 
    afinal_production_matrix_list.append(np.matmul(depreciation_list_input[T + 1], (
        supply_list_input[T] - (use_dom_list_input[T] + use_imp_list_input[T]))))
for T in range(time_steps_input): 
    v = deepcopy(np.matmul(depreciation_list_input[T + 1], target_out_list_input[T]))
    for i in range(planning_horizon_input - 1):
        w = deepcopy(np.concatenate((v, (
            np.asarray(np.matmul(depreciation_list_input[i + 2], target_out_list_input[i + 1]) + v[i])))))
        v = deepcopy(w)
    aexport_value_list = []
    for i in range(time_steps_input):
        aexp_val = 0
        for j in range(len(export_prices_list_input[i])):
            aexp_val += export_prices_list_input[i][j] * export_vector_list_input[i][j]
        aexport_value_list.append(aexp_val)
    atarget_output_aggregated = np.concatenate((v, [-aexport_value_list[T]]))
    atarget_output_aggregated_list.append(atarget_output_aggregated[:-1])

for T in range(time_steps_input):

    x = np.array_split(result_list[T], planning_horizon_input)

    l = np.array_split(lagrange_list[T], planning_horizon_input)

    y = []
    for i in range(planning_horizon_input):
        y.append(np.transpose(np.squeeze(np.array(np.matmul(afinal_production_matrix_list[T + i], x[i])))))

    tout_planning_period = np.array_split(atarget_output_aggregated_list[T], planning_horizon_input)
    for i in range(planning_horizon_input):
        overshoot.append((zdivide(y[i], np.squeeze(np.asarray(tout_planning_period[i])))) - 1)

# Displaying results of all plans in order

plt.style.use('_mpl-gallery')
fig = plt.figure(figsize=(30, 180))
fig.suptitle('Results', fontsize=32)
gs = gridspec.GridSpec(5 * time_steps_input * planning_horizon_input, 1)

    
sector_with_all_outputs_and_EXP = deepcopy(sector_with_all_outputs_input)
sector_with_all_outputs_and_EXP.append('EXP')

labels = ['overshoot_target_output_quotient',
            'Worked hours (10K)',
            'Worked hours (10K) percentage',
            'Produced total period minus consumed total',
            'Lagrange multiplier']

for i in range(time_steps_input):
    ax = fig.add_subplot(gs[i * 5 + 1, 0])
    ax.set_xlabel('Product', fontsize=14)
    ax.set_ylabel(labels[0], fontsize=14)
    ax.set_xticks(range(afinal_production_matrix_list[i].shape[0]), sector_with_all_outputs_input)
    for j in range(planning_horizon_input):
        ax.plot(range(afinal_production_matrix_list[j].shape[0]), overshoot[j])

    ax = fig.add_subplot(gs[i * 5 + 2, 0])
    ax.set_xlabel('Product', fontsize=14)
    ax.set_ylabel(labels[1], fontsize=14)
    ax.set_xticks(range(afinal_production_matrix_list[i].shape[1]), sector_name_input)
    for j in range(planning_horizon_input):
        x = np.array_split(result_list[i], planning_horizon_input)
        ax.plot(range(x[j].shape[0]), x[j])

    ax = fig.add_subplot(gs[i * 5 + 3, 0])
    ax.set_xlabel('Product', fontsize=14)
    ax.set_ylabel(labels[2], fontsize=14)
    ax.set_xticks(range(afinal_production_matrix_list[i].shape[1]), sector_name_input)
    for j in range(planning_horizon_input):
        x = np.array_split(result_list[i], planning_horizon_input)
        ax.plot(range(x[j].shape[0]), x[j] / worked_hours)

    ax = fig.add_subplot(gs[i * 5 + 4, 0])
    ax.set_xlabel('Product', fontsize=14)
    ax.set_ylabel(labels[3], fontsize=14)
    ax.set_xticks(range(len(sector_with_all_outputs_input)), sector_with_all_outputs_input)
    total_overshoot = 0.0
    for j in range(planning_horizon_input):
        total_overshoot += overshoot[j]
    ax.plot(range(afinal_production_matrix_list[i].shape[0]), total_overshoot)

    ax = fig.add_subplot(gs[i * 5 + 5, 0])
    ax.set_xlabel('Product', fontsize=14)
    ax.set_ylabel(labels[4], fontsize=14)
    ax.set_xticks(range(len(sector_with_all_outputs_and_EXP)), sector_with_all_outputs_and_EXP)
    for j in range(planning_horizon_input):
        l = np.array_split(lagrange_list[i], planning_horizon_input)
        ax.plot(range(l[j].shape[0]), l[j])

plt.show()
